In [141]:
import pandas as pd
df = pd.read_csv('compustat.csv')
df.head()

/var/folders/kp/rt42wzwx7sz9plr9l175fkv40000gn/T/ipykernel_96894/313954475.py:2: DtypeWarning: Columns (10,12,13,17,18,20,25,26,30,32,33,34,35,647,648,649,650,654,656,664,668,669,674,675,678,679) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('compustat.csv')


,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,1004,2000-02-29,1999,3.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
1,1004,2000-05-31,1999,4.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
2,1004,2000-08-31,2000,1.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
3,1004,2000-11-30,2000,2.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
4,1004,2001-02-28,2000,3.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01


In [142]:
df_with_ratios = pd.DataFrame(df[['datadate', 'gvkey', 'fqtr', 'fyr', 'tic']])
df_with_ratios.isna().sum()

datadate      0
gvkey         0
fqtr        346
fyr           0
tic         694
dtype: int64

Looks like gvkey would be a better primary key than ticker since it has no nans in this dataset. We also need to inspect financial quarter

In [143]:
df_with_ratios[df['fqtr'].isna()]

,datadate,gvkey,fqtr,fyr,tic
2746,2014-03-31,1224,NaN,12,EGN1
2747,2014-06-30,1224,NaN,12,EGN1
12300,2014-11-30,1951,NaN,8,PRSNQ
31199,2014-12-31,3424,NaN,9,BH
35114,2014-12-31,3806,NaN,9,DWSN
...,...,...,...,...,...
1050126,2013-06-30,196766,NaN,3,ALGDF
1050128,2013-09-30,196766,NaN,3,ALGDF
1074260,2015-06-30,317427,NaN,3,NOMD
1074262,2015-09-30,317427,NaN,3,NOMD


No clear pattern in financial quarters with nan values. Just use date value since there is no nans there

In [144]:
df_with_ratios.drop(['fqtr', 'fyr'], axis=1, inplace=True)
df_with_ratios.head()

,datadate,gvkey,tic
0,2000-02-29,1004,AIR
1,2000-05-31,1004,AIR
2,2000-08-31,1004,AIR
3,2000-11-30,1004,AIR
4,2001-02-28,1004,AIR


In [145]:
# Feature engineering
df_with_ratios['ta'] = df['atq']
df_with_ratios['lt'] = df['ltq']
df_with_ratios['dlc'] = df['dlcq']
df_with_ratios['dtp'] = df['txditcq']
df_with_ratios['che'] = df['cheq']
df_with_ratios['lct'] = df['lctq']
df_with_ratios['act'] = df['atq']
df_with_ratios['invtq'] = df['invtq']
df_with_ratios['intanq'] = df['intanq']
df_with_ratios['oiadp'] = df['oibdpq'] / df['oibdpy']
df_with_ratios['txp'] = df['txpq']
# df_with_ratios['xint'] = df['tieq'] / df['tiey']

In [146]:
df_with_ratios['pi'] = df['piq'] / df['piy']
df_with_ratios['ni'] = df['niq'] / df['niy']
df_with_ratios['rev'] = df['revtq'] / df['revty']
df_with_ratios['cogs'] = df['cogsq'] / df['cogsy']
df_with_ratios['oancf'] = df['oancfy']
df_with_ratios['ivncf'] = df['ivncfy']
df_with_ratios['fncfa'] = df['fincfy']
df_with_ratios['curr_ratio'] = df['actq'] / df['lctq']
df_with_ratios['quick_ratio'] = (df['actq'] - df['invtq']) / df['lctq']
df_with_ratios['cash_ratio'] = df['chechy'] / df['lctq']

In [147]:
df_with_ratios['net_working_capital'] = df['actq'] - df['lctq']
df_with_ratios['debt_ratio'] = df['dlttq'] / df['actq']
df_with_ratios['debt_equity_ratio'] = df['dlttq'] / df['ceqq']
df_with_ratios['equity_ratio'] = df['ceqq'] / df['actq']
df_with_ratios['financial_leverage_ratio'] = df['actq'] / df['ceqq']
# df_with_ratios['interest_coverage_ratio'] = (df['uopiq'] + df['dpq']) / df['tiey']
df_with_ratios['cashflow_debt_ratio'] = df['oancfy'] / df['dlttq']
df_with_ratios['net_profit_margin'] = 100 * (df['revtq'] - df['cogsq']) / df['revtq']
df_with_ratios['roa'] = (df['niq'] / df['atq']) * 100

In [148]:
df_with_ratios['asset_turnover'] = df['saleq'] / df['saley']
df_with_ratios['inventory_turnover'] = df['invchy']
df_with_ratios['inventory_days'] = df_with_ratios['inventory_turnover']
df_with_ratios['receivables_turnover'] = df['saleq'] / df['rectq']
df_with_ratios['dso'] = 365 / df_with_ratios['receivables_turnover']
df_with_ratios['working_capital_turnover'] = df['saleq'] / df['wcapq']
# df_with_ratios['debt_service_coverage_ratio'] = df['uopiq'] / df['dlttq']
# df_with_ratios['cash_coverage_ratio'] = df['oibdpq'] / df['tieq']
df_with_ratios['pe_ratio'] = df['prccq'] / df['epsf12']
# df_with_ratios['pb_ratio'] = df['prccq'] / (df['uceqq'] / df['cshoq'])
# df_with_ratios['div_payout_ratio'] = df['dvy'] / ((df['uniamiq'] * 100) / df['saley'])
df_with_ratios['retention_ratio'] = df['req'] / df_with_ratios['ni']
# df_with_ratios['ebitda'] = (df['uopiq'] + df['dpq'] * 100) / df['revtq']

In [149]:
df_with_ratios.head(5)

,datadate,gvkey,tic,ta,lt,dlc,dtp,che,lct,act,...,net_profit_margin,roa,asset_turnover,inventory_turnover,inventory_days,receivables_turnover,dso,working_capital_turnover,pe_ratio,retention_ratio
0,2000-02-29,1004,AIR,753.755,411.273,25.353,52.701,4.431,176.733,753.755,...,18.234795,1.453390,0.340731,-15.536,-15.536,1.606568,227.192332,0.777640,14.751553,603.096525
1,2000-05-31,1004,AIR,740.998,401.483,26.314,56.020,1.241,163.816,740.998,...,18.944015,0.333469,0.219732,-33.237,-33.237,1.753662,208.135899,0.647801,10.839844,2860.654487
2,2000-08-31,1004,AIR,747.543,408.290,48.151,58.441,0.669,168.282,747.543,...,16.176118,0.422584,1.000000,-8.260,-8.260,1.915678,190.533110,0.706938,11.138614,200.908000
3,2000-11-30,1004,AIR,772.941,431.677,50.437,60.937,3.069,189.367,772.941,...,18.857738,0.553470,0.466415,15.085,15.085,1.656685,220.319516,0.649498,13.474026,351.312198
4,2001-02-28,1004,AIR,754.718,409.853,49.665,60.695,0.767,167.852,754.718,...,20.259808,0.713909,0.306305,-13.415,-13.415,1.506264,242.321426,0.557790,23.859649,490.594335


In [150]:
print(f"length of dataset: {len(df_with_ratios)}")
print(f"shape of dataset: {df_with_ratios.shape}")

length of dataset: 1075571
shape of dataset: (1075571, 40)


In [151]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan'] = (df_with_ratios.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

,pct_nan
datadate,0.000000
gvkey,0.000000
tic,0.064524
ta,24.081441
lt,24.209559
dlc,28.118739
dtp,34.326511
che,24.327729
lct,36.964087
act,24.081441


In [139]:

# df_with_ratios['debt_service_coverage_ratio'] = df['uopiq'] / df['dlttq']
# df_with_ratios['xint'] = df['tieq'] / df['tiey']
# df_with_ratios['interest_coverage_ratio'] = (df['uopiq'] + df['dpq']) / df['tiey']
# df_with_ratios['cash_coverage_ratio'] = df['oibdpq'] / df['tieq']
# df_with_ratios['pb_ratio'] = df['prccq'] / (df['uceqq'] / df['cshoq'])
# df_with_ratios['div_payout_ratio'] = df['dvy'] / ((df['uniamiq'] * 100) / df['saley'])
# df_with_ratios['ebitda'] = (df['uopiq'] + df['dpq'] * 100) / df['revtq']

In [117]:
# Run this code to check the percentage of nan valus in the original compustat dataset for a column
# (len(df[df['dlttq'].isna() == True]) / len(df)) * 100

24.690048355710594

At this point our dataset should have columns with at most 20-30% nan values. we shall now inpute them using sector median. 
lets first try Methodology 1: 
1. map gvkey to sector
2. eg. if company X in sector A on date 2000-01-01 has a missing value for total income, we inpute it using the median of sector A in 2000-01-01. 

This assumes that on a particular date, a companies fundamentals can be represented by its sector median on that date